<div>
     <div>
        <img src="./report/isel_logo.png" width="400" height="400" align="left">
    </div>
    <div>
        <h2>Área Departamental de Engenharia de Eletrónica e Telecomunicações e de Computadores</h2>
        <p>Trabalho prático 4</p>
        <p>Autor:	44598	André L. A. Q. de Oliveira</p>
        <p>Unidade Curricular Compressão de Sinais Multimédia</p>
        <p>Professor: André Lourenço</p>
        <p>27 - Junho - 2021</p>
    </div>
</div>

### <a id="index"></a>

# Index
- [Tabelas da norma ITU T.81 - JPEG standard](#ITU_T.81)
- [Compressão](#compressao)
    - [snr](#snr)
- [Entropia](#entropia)
    - [entropy](#entropy)
    - [efficiency](#efficiency)
- [Utilities](#utilities)
    - [imshow](#imshow)
    - [convertFrame2Jpeg](#convertFrame2Jpeg)
    - [createPframe](#createPframe)
    - [mae](#mae)
    - [divide_to_blocks](#divide_to_blocks)
    - [merge_from_blocks](#merge_from_blocks)
- [Codificador 1](#codificador_1)
    - [encode_1](#encode_1)
- [Codificador 2](#codificador_2)
    - [encode_2](#encode_2)
    - [decode_2](#decode_2)
- [Codificador 3](#codificador_2)
    - [encode_3](#encode_3)
    - [decode_3](#decode_3)
- [Testes](#testes)
- [Conclusões](#conclusoes)

# Importar bibliotecas

In [276]:
import os
import math
from time import time
import cv2
import numpy as np
import matplotlib.pyplot as plt

cwd = os.getcwd() # current work diretory

<a id="compressao"></a>

# Compressão

A taxa de compressão pode ser cáculada atrás da seguinte expressão matemática:

$$ T_c = \frac{D_o}{D_c} $$

<a id="snr"></a>

## SNR

A relação sinal-ruído compara o nível de um sinal desejado com o nível do ruído. Quanto mais alta for a relação sinal-ruído, menor é o efeito do ruído de fundo sobre a detecção ou medição do sinal.

$$ SNR(Db) = 10\log_{10}\left[\frac{\sum_{l}\sum_{c}I_{ap}(l,c)^2}{\sum_{l}\sum_{c}[I_{ap}(l,c)-I_{or}(l,c)]^2}\right] $$

In [278]:
def snr(I_or, I_ap):
    Pxa = np.sum(I_ap.astype('float')**2)
    Pe = np.sum( (I_ap.astype('float') - I_or.astype('float'))**2 )
    return 10 * np.log10(Pxa / Pe)

<a id="entropia"></a>

# Entropia

A entropia mede a quantidade de informação codificada na mensagem, onde quando maior for o valor entrópico, maior será a incerteza.

A entropia da fonte é dada pela seguinte expressão matemática:

$$ H(S) = -\sum_{i=1}^{N}p(s_i)log_{2}p(s_i) $$


A eficiência da condificação pode ser obtida através da seguinte expressão matemática:

$$ \eta = \frac{H(S)}{L} $$

onde, **_L_** é o número médio de bits por símbolo.

É possível codificar uma fonte, sem perdas, se o número médio de bits por símbolo for maior ou igual à entropia da fonte:

$$ H(S) < L < H(S) + \delta $$

<a id="entropy"></a>

## entropy


In [279]:
def calculate_entropy(symbol_freq_dictionary):
    # list of symbol occurences in file
    occurrences = list(symbol_freq_dictionary.values())
    # total number of symbols
    t = np.sum(occurrences)
    # probability of each symbol
    p = [occ / t for occ in occurrences]
    return -np.sum([p * np.log2(p)])

<a id="efficiency"></a>

## efficiency

In [280]:
def efficiency(H, dictionary):
    # average number of bits per symbol
    L = 0
    for symbol in dictionary:
        L += len(dictionary[symbol])
    L = L / len(dictionary)
    return L, ( H / L )

[back to index](#index)

<a id="utilities"></a>

# Utilities

<a id="imshow"></a>

## imshow

In [281]:
def imshow(title, image):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

<a id="convertFrame2Jpeg"></a>

## convertFrame2Jpeg

In [282]:
def convertFrame2Jpeg(path, frame):
    cv2.imwrite(f"{path}.jpg", frame, (cv2.IMWRITE_JPEG_QUALITY, 50))
    return cv2.imread(f"{path}.jpg", cv2.IMREAD_GRAYSCALE)

<a id="create_Pframe"></a>

## create_Pframe

In [283]:
def create_Pframe(Iframe, frame):
    return ( ( Iframe.astype('int16') - frame.astype('int16') ) / 2  + 127 ).astype('uint8')

In [284]:
i =  np.random.randint(256, size=(8,8)).astype('uint8')
print(i)

f =  np.random.randint(256, size=(8,8)).astype('uint8')
print(f)

p = ( ( i.astype('int16') - f.astype('int16') ) / 2 ) + 127
# print(p)

p_ = ( i.astype('int16') - (p.astype('int16') - 127)*2).astype('uint8')
print(p_)

[[118 148 248  88 103 233  94  81]
 [ 17 126   5 234 119 106 237  43]
 [150  70 137 183 172 218  29 226]
 [214 227 211 126  79  88 146 244]
 [  8 137   7 133  85 247  35 100]
 [179  19  94 195 245 174  89 162]
 [235   3 116  58 246  59 231   3]
 [ 92  28 149  55  38  72 240  60]]
[[106 202  46 139 247 208 202 206]
 [ 60 188 239 185  60 199 243  44]
 [202 214  28 186 170 207 135 231]
 [ 16 205 158 169 146 149   8 233]
 [ 98  89 137  90 123 182  64 123]
 [ 60 127 117  88 182  16  87 205]
 [178 133 145  88 226 251 250 172]
 [137  16 144 156  45 233  61 137]]
[[106 202  46 140 247 209 202 207]
 [ 61 188 239 186  61 200 243  45]
 [202 214  29 187 170 208 135 232]
 [ 16 205 159 170 147 150   8 234]
 [ 98  89 137  91 123 183  65 124]
 [ 61 127 118  89 183  16  87 206]
 [179 133 146  88 226 251 251 173]
 [138  16 145 157  46 234  62 138]]


In [285]:
i_f = np.random.randint(256, size=(8,8)).astype('uint8')
# i_f = np.zeros((8,8)).astype('uint8')
print("i_f\n",i_f)
f = np.random.randint(256, size=(8,8)).astype('uint8')
# f = (np.ones((8,8))*255).astype('uint8')
print("f\n",f)
p_f = create_Pframe(i_f,f)
print("p_f\n",p_f)

i_f
 [[ 39 173  91  47  82 150 227  76]
 [236 216 169  16   8 171 151  61]
 [  1  33 171  21 134 254  23 208]
 [  5 202 211 135 193 102 232  65]
 [195 211 179 193 135 124 106 112]
 [ 54 210 183  32 160 181 208 209]
 [ 40  77 135 150 131 170 165  86]
 [195  79  89 227 137 116 116 167]]
f
 [[ 78 232  70  51 232 132 168 223]
 [121 144  31 197 141  12  77  26]
 [174 119 167  50 229 146 250 144]
 [ 14 130 191 226 200 142  26 216]
 [237 100 116  27 170 192  19 213]
 [ 47 178   7 140 162 253  29 177]
 [ 60 247 173 232  36 207  15  86]
 [ 78  16 254  11 244 135  91 125]]
p_f
 [[107  97 137 125  52 136 156  53]
 [184 163 196  36  60 206 164 144]
 [ 40  84 129 112  79 181  13 159]
 [122 163 137  81 123 107 230  51]
 [106 182 158 210 109  93 170  76]
 [130 143 215  73 126  91 216 143]
 [117  42 108  86 174 108 202 127]
 [185 158  44 235  73 117 139 148]]


<a id="reconstruct_Iframe"></a>

## reconstruct_Iframe

In [286]:
def reconstruct_Iframe(Iframe, Pframe):
    return ( Iframe.astype('int16') - (Pframe.astype('int16') - 127)*2).astype('uint8')

In [287]:
rf = reconstruct_Iframe(i_f,p_f)
print(rf)

[[ 79 233  71  51 232 132 169 224]
 [122 144  31 198 142  13  77  27]
 [175 119 167  51 230 146 251 144]
 [ 15 130 191 227 201 142  26 217]
 [237 101 117  27 171 192  20 214]
 [ 48 178   7 140 162 253  30 177]
 [ 60 247 173 232  37 208  15  86]
 [ 79  17 255  11 245 136  92 125]]


In [288]:
macro_blocks = np.zeros((8, 8), dtype=object)
macro_blocks[0][0] =  np.array([[1,2],
                       [1,2]])
print(macro_blocks.shape)

(8, 8)


<a id="mae"></a>

## Erro absoluto médio (MAE)

Critério de semelhança entre blocos

$$ (d_m,d_n) = arg \min_{d_m,d_n} \frac{1}{MN}\sum_{m=1}^{M}\sum_{n=1}^{N} \left| X^{t}(m,n)-X^{t-1}(m-d_m,n-d_n) \right| $$

In [289]:
def mae(b1, b2):
    return np.sum(np.abs(cv2.subtract(b1,b2).ravel())) / len(b1.ravel())

In [290]:
block1 = np.ones(256).reshape((16,16))
block2 = (np.ones(256)*3).reshape((16,16))

print(mae(block1, block2))

2.0


<a id="to_macro_blocks"></a>

### to_macro_blocks

Divide uma imagem em uma matriz de blocos de tamanho size x size.

In [291]:
def to_macro_blocks(image, size):
    
    width, height = image.shape
    
    if (width % size) != 0 or (height % size) != 0:
        raise Exception('function only supports multiple macro block division!')
    
    macro_blocks = np.zeros((int(width/size), int(height/size)), dtype=object)
    
    i = -1
    j = -1
    for lin in range(0, width, size):
        i += 1
        j = -1
        for col in range(0, height, size):
            j += 1
            # create new block
            macro_blocks[i][j] = image[lin:(lin + size),col:(col  +size)]
                
    return macro_blocks

<a id="from_macro_blocks"></a>

### from_macro_blocks

Merge um matriz de macro blocos para uma imagem.

In [292]:
def from_macro_blocks(matrix, size):
    
    width  = matrix.shape[0]*size
    heigth = matrix.shape[1]*size
    
    output = np.zeros((width, heigth), dtype='uint8')
    
    i = -1
    j = -1
    for lin in range(0, width, size):
        i+=1
        j=-1
        for col in range(0, heigth, size):
            j+=1
            output[lin:(lin+size),col:(col+size)] = matrix[i][j]
    return output

In [293]:
arr1 = np.arange(64).reshape((8,8))
m = to_macro_blocks(arr1, 4)
print(m)

[[array([[ 0,  1,  2,  3],
         [ 8,  9, 10, 11],
         [16, 17, 18, 19],
         [24, 25, 26, 27]]) array([[ 4,  5,  6,  7],
                                   [12, 13, 14, 15],
                                   [20, 21, 22, 23],
                                   [28, 29, 30, 31]])]
 [array([[32, 33, 34, 35],
         [40, 41, 42, 43],
         [48, 49, 50, 51],
         [56, 57, 58, 59]]) array([[36, 37, 38, 39],
                                   [44, 45, 46, 47],
                                   [52, 53, 54, 55],
                                   [60, 61, 62, 63]])]]


In [294]:
frame = np.arange(64).reshape((8,8))
print('frame\n', frame)

mBlocks = to_macro_blocks(frame, 4)
print('divite to blocks\n', mBlocks)

r_frame = from_macro_blocks(mBlocks,4)
print('merge from blocks\n', r_frame)

frame
 [[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]]
divite to blocks
 [[array([[ 0,  1,  2,  3],
         [ 8,  9, 10, 11],
         [16, 17, 18, 19],
         [24, 25, 26, 27]]) array([[ 4,  5,  6,  7],
                                   [12, 13, 14, 15],
                                   [20, 21, 22, 23],
                                   [28, 29, 30, 31]])]
 [array([[32, 33, 34, 35],
         [40, 41, 42, 43],
         [48, 49, 50, 51],
         [56, 57, 58, 59]]) array([[36, 37, 38, 39],
                                   [44, 45, 46, 47],
                                   [52, 53, 54, 55],
                                   [60, 61, 62, 63]])]]
merge from blocks
 [[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43

 <a id="codificador_1"></a>
 
# Codificador 1
 
<div class="alert alert-block alert-info">
Considera que cada frame é uma intra-frame (I).
</div>

In [295]:
def encode_1(path, frames):
    Iframes = []
    for i in range(len(frames)):
        Iframes.append( convertFrame2Jpeg(f"{path}/Iframe_{i+1}", frames[i]) )
    return Iframes

<a id="codificador_2"></a>
 
# Codificador 2
 
<div class="alert alert-block alert-info">
Considera que todas as frames à exceção da primeira (a I-frame) são inter-frames (P), sem compensação de movimento.
</div>

In [296]:
def encode_2(path, frames):
    Iframe = frames[0]
    Pframes = []
    convertFrame2Jpeg(f"{path}/Iframe_{1}", Iframe)
    for i in range(1, len(frames)):
        Pframes.append( convertFrame2Jpeg(f"{path}/Pframe_{i+1}", create_Pframe(frames[0], frames[i])) )
    return Iframe, Pframes

In [297]:
def decode_2(path, Iframe, Pframes):
    rIframes = []
    for i in range(len(Pframes)):
        rIframes.append( convertFrame2Jpeg(f"{path}/rIframe_{i+2}", reconstruct_Iframe(Iframe, Pframes[i])) )
    return Iframe, rIframes

<a id="codificador_3"></a>
 
# Codificador 3
 
<div class="alert alert-block alert-info">
Considera que todas as frames à exceção da primeira (a I-frame) são inter-frames (P), com compensação de movimento.
</div>

In [301]:
def encode_3(path, frames):
    
    output = []
    
    Iframe = frames[0]
    macroBlocks = []
    #for b in range(1, 2):
        # divide frame i into 16x16 macro blocks
    macroBlocks = to_macro_blocks(frames[1], 16)
        # for each macro block we search in (-15, 15) window for the best match of Iframe trought MAE
    s = ""
    print(macroBlocks.shape)
    for i in range(macroBlocks.shape[0]):
        for j in range(macroBlocks.shape[1]):
            ### bounderies
            # top
            if i == 0:
                if j == 0:
                    s += f"sob - ({i},{j}) "
                elif j == (macroBlocks.shape[1] - 1):
                    s += f"({i},{j})\n"
                else:
                    s += f"({i},{j}) "
            # bottom
            elif i == (macroBlocks.shape[0] - 1):
                if j == 0:
                    s += f"({i},{j}) "
                elif j == (macroBlocks.shape[1] - 1):
                    s += f"({i},{j}) - eob"
                else:
                    s += f"({i},{j}) "
            # left
            elif j == 0:
                s += f"({i},{j}) "
            # right
            elif j == (macroBlocks.shape[1] - 1):
                s += f"({i},{j})\n"
            # center
            else:
                s += f"({i},{j}) "
    print(s)
                          
   
    return macroBlocks

In [302]:
def best_match(Iframe, macroBlock):
    s = ""
    print(macroBlocks.shape)
    for i in range(macroBlocks.shape[0]):
        for j in range(macroBlocks.shape[1]):
            ### bounderies
            # top
            if i == 0:
                if j == 0:
                    s += f"sob - ({i},{j}) "
                elif j == (macroBlocks.shape[1] - 1):
                    s += f"({i},{j})\n"
                else:
                    s += f"({i},{j}) "
            # bottom
            elif i == (macroBlocks.shape[0] - 1):
                if j == 0:
                    s += f"({i},{j}) "
                elif j == (macroBlocks.shape[1] - 1):
                    s += f"({i},{j}) - eob"
                else:
                    s += f"({i},{j}) "
            # left
            elif j == 0:
                s += f"({i},{j}) "
            # right
            elif j == (macroBlocks.shape[1] - 1):
                s += f"({i},{j})\n"
            # center
            else:
                s += f"({i},{j}) "
    print(s)

(15, 22)
sob - (0,0) (0,1) (0,2) (0,3) (0,4) (0,5) (0,6) (0,7) (0,8) (0,9) (0,10) (0,11) (0,12) (0,13) (0,14) (0,15) (0,16) (0,17) (0,18) (0,19) (0,20) (0,21)
(1,0) (1,1) (1,2) (1,3) (1,4) (1,5) (1,6) (1,7) (1,8) (1,9) (1,10) (1,11) (1,12) (1,13) (1,14) (1,15) (1,16) (1,17) (1,18) (1,19) (1,20) (1,21)
(2,0) (2,1) (2,2) (2,3) (2,4) (2,5) (2,6) (2,7) (2,8) (2,9) (2,10) (2,11) (2,12) (2,13) (2,14) (2,15) (2,16) (2,17) (2,18) (2,19) (2,20) (2,21)
(3,0) (3,1) (3,2) (3,3) (3,4) (3,5) (3,6) (3,7) (3,8) (3,9) (3,10) (3,11) (3,12) (3,13) (3,14) (3,15) (3,16) (3,17) (3,18) (3,19) (3,20) (3,21)
(4,0) (4,1) (4,2) (4,3) (4,4) (4,5) (4,6) (4,7) (4,8) (4,9) (4,10) (4,11) (4,12) (4,13) (4,14) (4,15) (4,16) (4,17) (4,18) (4,19) (4,20) (4,21)
(5,0) (5,1) (5,2) (5,3) (5,4) (5,5) (5,6) (5,7) (5,8) (5,9) (5,10) (5,11) (5,12) (5,13) (5,14) (5,15) (5,16) (5,17) (5,18) (5,19) (5,20) (5,21)
(6,0) (6,1) (6,2) (6,3) (6,4) (6,5) (6,6) (6,7) (6,8) (6,9) (6,10) (6,11) (6,12) (6,13) (6,14) (6,15) (6,16) (6,17) (6,18

In [300]:
def sortDictionaryByDistance(d: dict,x: int, y: int):
    return dict(sorted(d.items(), key=lambda item: math.sqrt((x-item[0][0])**2+(y-item[0][1])**2)))

In [ ]:
def getMovementEstimationFrame(Iframe, dictionary: dict):

In [ ]:
d = dict()
d[(0,0)] = 3
d[(0,1)] = 7
d[(0,4)] = 6
d[(0,3)] = 2
d[(1,1)] = 2

print(d)
d = sortDictionaryByDistance(d, 1, 3)
print(d)

best_key = min(d, key=d.get)
print(best_key)

In [ ]:
math.sqrt((0-1)**2+(0-2)**2)

In [ ]:
def decode_3(path, frames):

### <a id="testes"></a>

# Testes

In [ ]:
error() # cause erros to stop here

## Importar dados

In [ ]:
# sorting frames names by alphabetical order
# frames[0] = sorted(frames[0], key=lambda x: (x[0],int(x[5:])) )

In [ ]:
frames = []
for i in range(len(os.listdir(f"{cwd}/input_data/bola_seq"))):
    frames.append( cv2.imread(f"{cwd}/input_data/bola_seq/bola_{i+1}.tiff", cv2.IMREAD_GRAYSCALE) )

<div class="alert alert-block alert-info">
Codificador 1
</div>

In [ ]:
Iframe1 = encode_1(f"{cwd}/output_data/codificador1", frames)

<div class="alert alert-block alert-info">
Codificador 2
</div>

In [ ]:
Iframe2, Pframes2 = encode_2(f"{cwd}/output_data/codificador2/encode", frames)

In [ ]:
Iframes2, rIframes2 = decode_2(f"{cwd}/output_data/codificador2/decode", Iframe2, Pframes2)

<div class="alert alert-block alert-info">
Codificador 3
</div>

[back to index](#index)

<a id="conclusoes"></a>

# Conclusões

Este trabalho explora os princípios básicos da codificação de vídeo. Neste trabalho pretende-se implementar três formas
de codificação de vídeo. Cada um destes codificadores deve ser testado com a a sequência de imagens disponibilizadas

[back to index](#index)